In [1]:
import pandas as pd
import requests
from collections import defaultdict
from pathlib import Path

In [2]:
year = 2021
leaderboard_id = 976545
token_path = Path.home() / '.config/aocd/token'

In [3]:
with open(token_path) as f:
    token = f.read().strip()

cookies = {'session': token}
response = requests.get(f'https://adventofcode.com/{year}/leaderboard/private/view/{leaderboard_id}.json', cookies=cookies)
j = response.json()

In [4]:
time_diffs = defaultdict(lambda: defaultdict(str))

for id, member in j['members'].items():
    name = member['name'] if member['name'] else id
    for day, times in member['completion_day_level'].items():
        if '1' in times and '2' in times:
            time_diffs[name][int(day)] = int(times['2']['get_star_ts']) - int(times['1']['get_star_ts'])

In [5]:
df = pd.DataFrame.from_dict(time_diffs, orient='index', dtype='Int64')
df.sort_index(key=lambda x: x.str.lower(), inplace=True)
df.sort_index(axis=1, inplace=True)

In [6]:
df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
1014573,963,246,1933,1398,9323,4422,1144,20172,<NA>,2862,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1047131,<NA>,245,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1139456,808,151,16775,583,274,21154,1619,13695,4179,8755,243,150,161,25,277,156,<NA>,<NA>,<NA>,<NA>
1495816,4343,219,<NA>,<NA>,<NA>,<NA>,210,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1525642,1379,240,2014,1157,6370,3167,387,6732,6978,1374,219,<NA>,2948,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1547063,51612,316,317896,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1550306,463141,264,5009,584746,<NA>,<NA>,280,31936,96499,1990,870,1140,275,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1603729,135,655,9390,1079,874,26,690,6927,1784,933,392,1098,578,471,5354,2077,<NA>,<NA>,<NA>,<NA>
1614144,16,1407,65,92,<NA>,<NA>,<NA>,9,799,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1625081,1848,553,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [7]:
num_members, num_days = df.shape
points = pd.Series(index=df.index, dtype=int)

for day in range(num_days):
    points_awarded = num_members
    for name, time in df[day+1].sort_values().iteritems():
        if not pd.isna(time):
            points[name] += points_awarded
            points_awarded -= 1

In [8]:
# part 2 leaderboard
points.sort_values(ascending=False).head(20)

zoidberg77          1060
Sanjay              1056
attoPascal          1041
NotsoblackReaper     969
michalrzak           953
KaNaDaAT             874
Mathematiker         844
David Wild           767
marc1729             746
Katakompe            724
S-ecki               702
1603729              700
1139456              693
underway             687
Zsivony1es           678
RaphaelTarita        672
Hainkinho            624
Yorarasu             600
st1fado              591
devgertschi          590
dtype: int64

In [9]:
# seconds for each part 2
df.loc['attoPascal']

1      238
2      211
3      773
4      812
5     1011
6      835
7      548
8     4083
9     1633
10     718
11     332
12     480
13     133
14    1548
15     611
16     440
17     196
18    1139
19     202
20     201
Name: attoPascal, dtype: Int64

In [10]:
# seconds for latest part 2
df.iloc[:, -1].dropna().sort_values()

michalrzak           28
Mathematiker        105
Sanjay              120
attoPascal          201
marc1729            242
NotsoblackReaper    347
Name: 20, dtype: Int64

In [11]:
# sum for all parts 2 in hours
df.sum(axis=1, skipna=False).dropna().sort_values() / 3600

attoPascal          4.48444
NotsoblackReaper    7.03806
Sanjay              8.18972
michalrzak          8.28389
dtype: object